/databricks-datasets/definitive-guide/data

In [0]:
# Creating a SparkSession in Python
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("Word Count")\
    .config("spark.some.config.option", "some-value")\
    .getOrCreate()


In [0]:
df1 = spark.range(2, 10000000, 2)
df2 = spark.range(2, 10000000, 4)
step1 = df1.repartition(5)
step12 = df2.repartition(6)
step2 = step1.selectExpr("id * 5 as id")
step3 = step2.join(step12, ["id"])
step4 = step3.selectExpr("sum(id)")

step4.collect() # 2500000000000


Out[2]: [Row(sum(id)=2500000000000)]

In [0]:
step4.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=true
+- == Final Plan ==
   *(5) HashAggregate(keys=[], functions=[finalmerge_sum(merge sum#21L) AS sum(id#14L)#17L])
   +- ShuffleQueryStage 3, Statistics(sizeInBytes=128.0 B, rowCount=8, isRuntime=true)
      +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [id=#405]
         +- *(4) HashAggregate(keys=[], functions=[partial_sum(id#14L) AS sum#21L])
            +- *(4) Project [id#14L]
               +- *(4) SortMergeJoin [id#14L], [id#8L], Inner
                  :- Sort [id#14L ASC NULLS FIRST], false, 0
                  :  +- AQEShuffleRead coalesced
                  :     +- ShuffleQueryStage 2, Statistics(sizeInBytes=76.3 MiB, rowCount=5.00E+6, isRuntime=true)
                  :        +- Exchange hashpartitioning(id#14L, 200), ENSURE_REQUIREMENTS, [id=#239]
                  :           +- *(3) Project [(id#6L * 5) AS id#14L]
                  :              +- ShuffleQueryStage 0, Statistics(sizeInBytes=76.3 MiB, rowCount=5